<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [12]</a>'.</span>

In [1]:
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import pandas as pd
import json
import tqdm
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset, DatasetDict
import evaluate
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
NUM_EPOCHS = 10
EXPERIMENT_NAME = "t5-small_falcon2-default_annotation-k5"
EXPERIMENT_DIR = Path('experiments')
MODEL_ARTIFACTS = EXPERIMENT_DIR / EXPERIMENT_NAME
WEIGHTS_DIR = MODEL_ARTIFACTS / 'weights'
VALS_DIR = MODEL_ARTIFACTS / 'validations'

Make appropriate directoreis

In [2]:
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)
VALS_DIR.mkdir(parents=True, exist_ok=True)

Defining the model and tokenizer

In [3]:
model_path = "t5-small"
tokenizer_path = "t5-small"

model = T5ForConditionalGeneration.from_pretrained(model_path, device_map ='auto')
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

Define dataset maker

In [4]:
def split_dataframe(df):
  # ratios from Bannerjee
  train = 0.7
  dev = 0.1
  test = 0.2
  assert train + dev + test == 1.0
  data_len = len(df)
  train_set = Dataset.from_pandas(df[:round(data_len * train)])
  dev_set = Dataset.from_pandas(df[round(data_len * train):round(data_len* (train + dev))])
  test_set = Dataset.from_pandas(df[round(data_len * (train + dev)):])
  
  dataset = DatasetDict()
  dataset['train'] = train_set
  dataset['dev'] = dev_set
  dataset['test'] = test_set

  return dataset

Define dataset tokenizer

In [5]:
def tokenize_data(dataset, column):
  print("tokenizing")
  model_inputs = tokenizer(dataset[column], padding=True, return_tensors="pt")
  return model_inputs

Define unmasker

In [6]:
from pipeline import T5Converter
converter = T5Converter()

Defining the validation function

In [7]:
def val(val_dataloader, val_path = None):
  model.eval()
  eval_dict = []

  iters = len(val_dataloader)

  progress_bar = tqdm.tqdm(iters)
  progress_bar.set_description(f"Eval")

  for val_batch in val_dataloader:
    batch = {}
    for k,v in val_batch.items():
      if k in {"input_ids", "labels", "attention_mask"}:
        batch[k] = v.to("cuda")
    
    with torch.no_grad():
      outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    for i, pred in enumerate(tokenizer.batch_decode(predictions)):
      gold = val_batch['gold'][i]
      gold = gold.replace(" ","")
      gold2 = gold.replace(">", "> ").replace("<"," <").replace("  ", " ").strip()
      pred = pred.replace(" ","").replace("</s>", "").replace("<pad>","")
      pred2 = pred.replace(">", "> ").replace("<"," <").replace("  ", " ").strip()
      eval_dict.append({
        "Utte": val_batch['utterance'][i],
        "Anno": val_batch['annotated'][i],
        "Gold": val_batch['gold'][i],
        "Gene": pred, # THIS NEEDS TO BE UNMASKED
        "Gol2": converter._unmask_generic(gold2),
        "Gen2": converter._unmask_generic(pred2),
      })
    progress_bar.update(1)
  
  if val_path:
    with open(val_path, "w") as f:
      json.dump(eval_dict, f, indent=2)
  
  model.train()
  return eval_dict

In [8]:
import time

def training_loop(df):
  print("beginning training")

  assert 'utterance' in df.columns
  assert 'annotated' in df.columns
  assert 'gold' in df.columns

  dataset = split_dataframe(df)
  tokenized_dataset = dataset \
    .map(lambda x: tokenize_data(x, 'gold'), batched=True) \
    .rename_column('input_ids', 'labels') \
    .map(lambda x: tokenize_data(x, 'annotated'), batched=True)

  tokenized_dataset.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)
  print("data loaded")
  
  train_dataset = tokenized_dataset["train"]
  dev_dataset = tokenized_dataset["dev"]
  test_dataset = tokenized_dataset["test"]

  train_dataloader = DataLoader(train_dataset, batch_size = 1)
  dev_dataloader = DataLoader(dev_dataset, batch_size = 1)

  scalar = 0
  i = 0

  optimizer = optim.AdamW(model.parameters(), lr = 0.0015)
  lr_scheduler=transformers. \
    get_polynomial_decay_schedule_with_warmup(optimizer, 5000, 30000, power=0.5)

  for epoch in range(NUM_EPOCHS):
    print("Beginning Epoch:", epoch)
    i = 0
    iters = len(train_dataloader)
    for batch in train_dataloader:
      newbatch = {}
      for k,v in batch.items():
        if k in ["labels", "input_ids", "attention_mask"]:
          newbatch[k] = v.to("cuda")
      
      batch = newbatch
      newbatch = {}

      outputs = model(**batch)
      loss = outputs.loss
      scalar += loss.mean().item()

      if (i + 1) % 100 == 0:
        print(f'iteration = {i + 1}/{iters}, training loss={scalar/100}')
        scalar = 0

      loss /= 10 
      loss.mean().backward()
      if (i+1) % 10:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
      
      del batch
      i += 1
    
    val(dev_dataloader, VALS_DIR / f"val_{epoch}.json")

    torch.save(model.state_dict(),
      WEIGHTS_DIR / f"cp_{epoch}.pth")

In [9]:
df_json = []
with open('falcon_links/2/link_24066.json') as f:
  data_json = json.load(f)

print(data_json[0])


[{'utterance': 'What is Delta Air Lines periodical literature mouthpiece?', 'ents': [{'uri': 'http://www.wikidata.org/entity/Q188920', 'prefix': 'wd:', 'id': 'Q188920'}, {'uri': 'http://www.wikidata.org/entity/Q1002697', 'prefix': 'wd:', 'id': 'Q1002697'}, {'uri': 'http://www.wikidata.org/entity/Q416938', 'prefix': 'wd:', 'id': 'Q416938'}, {'uri': 'http://www.wikidata.org/entity/Q523753', 'prefix': 'wd:', 'id': 'Q523753'}, {'uri': 'http://www.wikidata.org/entity/Q671722', 'prefix': 'wd:', 'id': 'Q671722'}], 'rels': []}, {'utterance': 'What is Delta Air Lines periodical literature mouthpiece?', 'fragments': ['[DEF]', 'wd:', 'Q188920 Delta', '[DEF]', 'wd:', 'Q1002697 periodical literature', '[DEF]', 'wd:', 'Q416938 Mouthpiece', '[DEF]', 'wd:', 'Q523753 mouthpiece', '[DEF]', 'wd:', 'Q671722 mouthpiece']}, {'inputs': 'What is Delta Air Lines periodical literature mouthpiece? <extra_id_59> <extra_id_53> Q188920 Delta <extra_id_59> <extra_id_53> Q1002697 periodical literature <extra_id_59> <

Main

In [10]:
# df_json = []
# with open('weekend.json') as f:
#   data_json = json.load(f)

for data in data_json:
  data_dict = {
    "utterance": data[0]["utterance"],
    "annotated": data[2]["inputs"],
    "gold": data[2]["labels"]
  }
  df_json.append(data_dict)

In [11]:
df = pd.DataFrame.from_dict(df_json)
df.head()

,utterance,annotated,gold
0,What is Delta Air Lines periodical literature ...,What is Delta Air Lines periodical literature ...,<extra_id_6> <extra_id_21> <extra_id_39> <extr...
1,What is the name of Ranavalona Is husbands child?,What is the name of Ranavalona Is husbands chi...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...
2,Are Jeff Bridges and Lane Chandler both photog...,Are Jeff Bridges and Lane Chandler both photog...,<extra_id_4> <extra_id_19> <extra_id_33> <extr...
3,What range are the papers at the Monique Genon...,What range are the papers at the Monique Genon...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...
4,Which is the operating income for Qantas?,Which is the operating income for Qantas? <ext...,<extra_id_6> <extra_id_21> <extra_id_39> <extr...


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [12]:
training_loop(df)

beginning training


Map:   0%|          | 0/16845 [00:00<?, ? examples/s]

tokenizing


tokenizing


tokenizing


...

Eval: : 2403it [01:38, 24.51it/s]

Eval: : 2406it [01:38, 24.53it/s]

Eval: : 2406it [01:39, 24.28it/s]

Beginning Epoch: 1


OutOfMemoryError: CUDA out of memory. Tried to allocate 582.00 MiB (GPU 0; 10.92 GiB total capacity; 9.20 GiB already allocated; 274.50 MiB free; 9.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF